In [1]:
#firt part of the project 
import pandas as pd
import networkx as nx
import time

# Step 1: Read the CSV file
file = pd.read_csv('soc-sign-bitcoinotc.csv') 

# Step 2: Extract columns as source, target, and weight
source = file.iloc[:, 0].tolist() # Extract first column data as source
target = file.iloc[:, 1].tolist() # Extract second column data as target
weights = file.iloc[:, 2].tolist() # Extract third column data as weights

# Step 3: Create a graph
G = nx.Graph()
for i in range(len(source)):
    G.add_edge(source[i], target[i], weight=weights[i])

# Step 4: Compute Minimum Spanning Tree
start_time = time.time()
mst = nx.minimum_spanning_tree(G)
total_cost = sum([edge[2]['weight'] for edge in mst.edges(data=True)])
execution_time = time.time() - start_time
print(f"Total Cost of Minimum Spanning Tree: {total_cost}")
print(f"Execution Time: {execution_time} seconds")

Total Cost of Minimum Spanning Tree: -5174
Execution Time: 0.27806878089904785 seconds


In [17]:
#second part of project
import csv
import timeit
import threading
import networkx as nx
from sklearn.cluster import KMeans

# Read the CSV file
csv_file_path = 'soc-sign-bitcoinotc.csv'
source_col = []
target_col = []
rating_col = []

with open(csv_file_path, mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # Skip the header row
    for row in csv_reader:
        source_col.append(row[0])
        target_col.append(row[1])
        rating_col.append(int(row[2]))

# Create a feature matrix (combine source and target columns)
feature_matrix = [[s, t] for s, t in zip(source_col, target_col)]

# Perform K-means clustering
n_clusters = 100
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(feature_matrix)

# Assign cluster labels to each data point
for i, label in enumerate(cluster_labels):
    source_col[i] = f"{source_col[i]}_cluster{label}"
    target_col[i] = f"{target_col[i]}_cluster{label}"

# Compute MST for each cluster
def compute_mst(cluster_source, cluster_target, cluster_rating):
    G = nx.Graph()
    for s, t, w in zip(cluster_source, cluster_target, cluster_rating):
        G.add_edge(s, t, weight=w)

    mst = nx.minimum_spanning_tree(G)
    return mst

# Create threads for parallel processing
threads = []
start_time = timeit.default_timer()

for i in range(n_clusters):
    thread = threading.Thread(target=compute_mst, args=(source_col[i::n_clusters], target_col[i::n_clusters], rating_col[i::n_clusters]))
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

end_time = timeit.default_timer()
execution_time = end_time - start_time

# Calculate cost for each cluster
cluster_costs = {}
for i in range(n_clusters):
    cluster_ratings = [r for r, label in zip(rating_col, cluster_labels) if label == i]
    cluster_costs[f"Cluster {i}"] = sum(cluster_ratings)

# Print cluster costs
for cluster, cost in cluster_costs.items():
    print(f"{cluster}: Cost = {cost}")
#-------------Third part of project--------------
# Create a function to merge two MSTs based on their lowest weighted edge
def merge_msts(msts):
    final_mst = nx.Graph()
    for mst in msts:
        for edge in mst.edges(data=True):
            source, target, weight = edge
            if final_mst.has_edge(source, target):
                if weight['weight'] < final_mst[source][target]['weight']:
                    final_mst[source][target]['weight'] = weight['weight']
            else:
                final_mst.add_edge(source, target, weight=weight['weight'])
    return final_mst
# Merge the resulting MST trees together
mst_trees = []
for i in range(n_clusters):
    mst = compute_mst([source_col[j] for j in range(i, len(source_col), n_clusters)],
                      [target_col[j] for j in range(i, len(target_col), n_clusters)],
                      [rating_col[j] for j in range(i, len(rating_col), n_clusters)])
    mst_trees.append(mst)

final_mst = merge_msts(mst_trees)


# Calculate cost of the final MST
final_cost = sum(rating_col)
print(f"Final MST cost: {final_cost}")

# Print complete execution time
print(f"Complete execution time: {execution_time:.4f} seconds")

Cluster 0: Cost = 295
Cluster 1: Cost = 1215
Cluster 2: Cost = -573
Cluster 3: Cost = 6
Cluster 4: Cost = 429
Cluster 5: Cost = 3232
Cluster 6: Cost = 354
Cluster 7: Cost = 350
Cluster 8: Cost = 915
Cluster 9: Cost = 695
Cluster 10: Cost = 277
Cluster 11: Cost = 236
Cluster 12: Cost = 1445
Cluster 13: Cost = 498
Cluster 14: Cost = 351
Cluster 15: Cost = 45
Cluster 16: Cost = 55
Cluster 17: Cost = 71
Cluster 18: Cost = 175
Cluster 19: Cost = 753
Cluster 20: Cost = -426
Cluster 21: Cost = -109
Cluster 22: Cost = 108
Cluster 23: Cost = 843
Cluster 24: Cost = 209
Cluster 25: Cost = 1327
Cluster 26: Cost = 140
Cluster 27: Cost = -2435
Cluster 28: Cost = 160
Cluster 29: Cost = 576
Cluster 30: Cost = 175
Cluster 31: Cost = 437
Cluster 32: Cost = 218
Cluster 33: Cost = 226
Cluster 34: Cost = -96
Cluster 35: Cost = 414
Cluster 36: Cost = -949
Cluster 37: Cost = 113
Cluster 38: Cost = 104
Cluster 39: Cost = 709
Cluster 40: Cost = 630
Cluster 41: Cost = 703
Cluster 42: Cost = 187
Cluster 43: Cost

In [ ]:
"comparison of total cost of MST and thier execution time between part 1 and part 2:"
#the total cost of the first part of project is less than the second part , because its final MST cost is calcalate based on the
#individual weights from the CSV file 
#in the second part of the project , the total cost of the MST is calculated based on all the edge weights form the dataset
#the execution time of the first part of the project is less than the second part, because When the MST algorithm is applied to 
#one individual graph, all the edges and vertices are considered together without any constraints or boundaries. This means that 
#the algorithm can find the minimum spanning tree by exploring all possible edges and vertices without any limitations.
#On the other hand, when the graph is divided into different clusters, the algorithm needs to consider constraints and boundaries
#between the clusters. This adds complexity to the algorithm as it needs to determine how to connect the different clusters in 
#a way that minimizes the overall cost while respecting the cluster boundaries.
#As a result, the time execution of the MST algorithm may be longer when the graph is divided into different clusters compared
#to when it is one individual graph because of the increased complexity and constraints involved in finding the MST.
